## DATA604 FINAL PROJECT | Toll Plaza Simulation

### Introduction

Paying toll to be able to travel to one's destination is largely viewed as an inconvenience.  Americans have, for the most part, come to view having free, open roads as an inalienable right from our government.  Toll roads are then aberrant and annoying. But  the  vexing  aspects  of  toll  roads  do  not  stop  at  the  quarter  or  35-cent  fee,  but rather include the time that drivers are forced to waste. Drivers have places to be and people to see, but for one reason or another, tolls must at times be collected from them.  Our goal in this project is to make the process more optimal for everyone involved, including the owners and operators of the booths, and of  course  the  drivers.    The  only  mechanism  for  optimization  at  our  disposal  is, presumably, adjustment of the number of booths present at a certain toll plaza, given the number of lanes entering and exiting it.

###  Problem Statement and Assumptions

Suppose a state highway department is planning a new toll exit for an existing turnpike. The number of toll booths to put at the exit is in question. The department wants to keep costs low by having as few booths as possible. but if the waiting lines get too long during rush hour and other peak periods it will hurt public relations, reduce the number of people who will use the exit, and, in the worst case, back waiting vehicles onto the highway-an unacceptable and potentially hazardous situation. The highway department believes that no more than six cars-on average across the lines-should be stored in the waiting lines during rush hour, but is willing to examine other average waiting-line lengths.

For obvious reasons, we don’t want to build too many toll booths unnecessarily, and we don’t want to build too few since that will jam the traffic. We describe the situation and how drivers behave when they approach the toll plaza in the following : 

1. Suppose that there are n toll booths in a toll plaza. For the safety reason, in front of each toll booth, we build a track for as many as 6 cars to wait in the line for paying toll, and once a car has driven into a track, the car can’t back up and change to another track.

2. For any car arriving the toll plaza, the driver always drives to a free booth, if any, and pays the tollright away without waiting in the line.

3. If no free booth available, the driver always selects a booth with the fewest cars in the track andget into the line (track). If there are more than one track with the same smallest number of cars inthem, the arriving driver will randomly pick up one. Because each car in the line may need different amount of time to pay the toll and the driver behind can’t predict this in advance, getting into the shortest line can’t guarantee the car can pass the booth faster than getting into a longer line, but the drive can’t do anything about it.

4. If all n tracks are full, every track already has 6 cars waiting, the newly arrived cars will be waiting in line on the open road for next available track. We can consider the capacity of the open road infinite. If some track become available, the car at the front end of the open road will get into the track. We assume there is only one line in the open road. If there are more one track become available, the driver will apply the same strategy to select a booth, i.e., going into the one with the shortest line.

### Simulation Process

In [13]:
import itertools
from collections import defaultdict

import random
import numpy as np
import pandas as pd
import math
import time

import simpy

import json

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt

import tkinter as tk
from PIL import ImageTk

In [ ]:
# -------------------------
#  CONFIGURATION
# -------------------------

CAR_ARRIVAL_MEAN = 3

TOLL_PAYMENT_MEAN = 0.4

TIME_TO_DRIVE_TO_BOOTH_MEAN = 1
TIME_TO_DRIVE_TO_BOOTH_STD = 0.25
TIME_TO_DRIVE_TO_EZPASS_MEAN = 0.5
TIME_TO_DRIVE_TO_EZPASS_STD = 0.1

TOLL_LINES = 4
TOLL_MEAN = 1
TOLL_STD = 0.2

EZPASS_LINES = 2
EZPASS_MEAN = 1 / 20
EZPASS_STD = 0.01


In [ ]:
# Let's pre-generate all the car arrival times 
# change the configuration, we'll have consistent arrivals
random.seed(42)
ARRIVALS = [ random.expovariate(1 / CAR_ARRIVAL_MEAN) for _ in range(40) ]

In [ ]:
# -------------------------
#  ANALYTICAL GLOBALS
# -------------------------

arrivals = defaultdict(lambda: 0)
toll_waits = defaultdict(lambda: [])
ezpass_waits = defaultdict(lambda: [])
event_log = []


def booth_arrivals(time, num):
    arrivals[int(time)] += num
    
def register_seller_wait(time, wait):
    toll_waits[int(time)].append(wait)